In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import numpy
from six.moves import xrange  # pylint: disable=redefined-builtin

from tensorflow.contrib.learn.python.learn.datasets import base
from tensorflow.python.framework import dtypes

from sklearn.decomposition import PCA

#loading weight file into variable
weights_file = np.load('weights.npz')
w1_file = weights_file['w1']
w2_file = weights_file['w2']
print(w1_file.shape) 

print(w2_file.shape)

(784, 30)
(30, 10)


In [0]:
#########for emnist-letters########
# from https://github.com/tensorflow/tensorflow/blob/7c36309c37b04843030664cdc64aca2bb7d6ecaa/tensorflow/contrib/learn/python/learn/datasets/mnist.py#L160
# modified for 10 classes of LETTERS
def _read32(bytestream):
  dt = numpy.dtype(numpy.uint32).newbyteorder('>')
  return numpy.frombuffer(bytestream.read(4), dtype=dt)[0]


def extract_images(f):
  """Extract the images into a 4D uint8 numpy array [index, y, x, depth].
  Args:
    f: A file object that can be passed into a gzip reader.
  Returns:
    data: A 4D uint8 numpy array [index, y, x, depth].
  Raises:
    ValueError: If the bytestream does not start with 2051.
  """
  print('Extracting', f.name)
  with gzip.GzipFile(fileobj=f) as bytestream:
    magic = _read32(bytestream)
    if magic != 2051:
      raise ValueError('Invalid magic number %d in MNIST image file: %s' %
                       (magic, f.name))
    num_images = _read32(bytestream)
    rows = _read32(bytestream)
    cols = _read32(bytestream)
    buf = bytestream.read(rows * cols * num_images)
    data = numpy.frombuffer(buf, dtype=numpy.uint8)
    data = data.reshape(num_images, rows, cols, 1)
    return data


def dense_to_one_hot(labels_dense, num_classes):
  labels_dense = labels_dense - 1 #ADDED THIS BC INDEX STARTS AT 1
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = numpy.arange(num_labels) * num_classes
  labels_one_hot = numpy.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot


def extract_labels(f, one_hot=False, num_classes=26):
  """Extract the labels into a 1D uint8 numpy array [index].
  Args:
    f: A file object that can be passed into a gzip reader.
    one_hot: Does one hot encoding for the result.
    num_classes: Number of classes for the one hot encoding.
  Returns:
    labels: a 1D uint8 numpy array.
  Raises:
    ValueError: If the bystream doesn't start with 2049.
  """
  print('Extracting', f.name)
  with gzip.GzipFile(fileobj=f) as bytestream:
    magic = _read32(bytestream)
    if magic != 2049:
      raise ValueError('Invalid magic number %d in MNIST label file: %s' %
                       (magic, f.name))
    num_items = _read32(bytestream)
    buf = bytestream.read(num_items)
    labels = numpy.frombuffer(buf, dtype=numpy.uint8)
    if one_hot:
      return dense_to_one_hot(labels, num_classes)
    return labels


class DataSet(object):

  def __init__(self,
               images,
               labels,
               fake_data=False,
               one_hot=False,
               dtype=dtypes.float32,
               reshape=True):
    """Construct a DataSet.
    one_hot arg is used only if fake_data is true.  `dtype` can be either
    `uint8` to leave the input as `[0, 255]`, or `float32` to rescale into
    `[0, 1]`.
    """
    dtype = dtypes.as_dtype(dtype).base_dtype
    if dtype not in (dtypes.uint8, dtypes.float32):
      raise TypeError('Invalid image dtype %r, expected uint8 or float32' %
                      dtype)
    if fake_data:
      self._num_examples = 10000
      self.one_hot = one_hot
    else:
      assert images.shape[0] == labels.shape[0], (
          'images.shape: %s labels.shape: %s' % (images.shape, labels.shape))
      self._num_examples = images.shape[0]

      # Convert shape from [num examples, rows, columns, depth]
      # to [num examples, rows*columns] (assuming depth == 1)
      if reshape:
        assert images.shape[3] == 1
        images = images.reshape(images.shape[0],
                                images.shape[1] * images.shape[2])
      if dtype == dtypes.float32:
        # Convert from [0, 255] -> [0.0, 1.0].
        images = images.astype(numpy.float32)
        images = numpy.multiply(images, 1.0 / 255.0)
    self._images = images
    self._labels = labels
    self._epochs_completed = 0
    self._index_in_epoch = 0

  @property
  def images(self):
    return self._images

  @property
  def labels(self):
    return self._labels

  @property
  def num_examples(self):
    return self._num_examples

  @property
  def epochs_completed(self):
    return self._epochs_completed

  def next_batch(self, batch_size, fake_data=False):
    """Return the next `batch_size` examples from this data set."""
    if fake_data:
      fake_image = [1] * 784
      if self.one_hot:
        fake_label = [1] + [0] * 9
      else:
        fake_label = 0
      return [fake_image for _ in xrange(batch_size)], [
          fake_label for _ in xrange(batch_size)
      ]
    start = self._index_in_epoch
    self._index_in_epoch += batch_size
    if self._index_in_epoch > self._num_examples:
      # Finished epoch
      self._epochs_completed += 1
      # Shuffle the data
      perm = numpy.arange(self._num_examples)
      numpy.random.shuffle(perm)
      self._images = self._images[perm]
      self._labels = self._labels[perm]
      # Start next epoch
      start = 0
      self._index_in_epoch = batch_size
      assert batch_size <= self._num_examples
    end = self._index_in_epoch
    return self._images[start:end], self._labels[start:end]

def remove_data(labels, images):
  labels_10 = []
  images_10 = []
  
  for i in range(len(labels)): #rows
    for j in range(0,10): #columns
      if labels[i][j] == 1:
        labels_10.append(labels[i][0:10])
        images_10.append(images[i])
  
  labels_10 = np.asarray(labels_10)
  images_10 = np.asarray(images_10)

  return labels_10, images_10
  
def read_data_sets(train_dir,
                   fake_data=False,
                   one_hot=False,
                   dtype=dtypes.float32,
                   reshape=True,
                   validation_size=5000):
  if fake_data:

    def fake():
      return DataSet([], [], fake_data=True, one_hot=one_hot, dtype=dtype)

    train = fake()
    validation = fake()
    test = fake()
    return base.Datasets(train=train, validation=validation, test=test)

  TRAIN_IMAGES = 'emnist-letters-train-images-idx3-ubyte.gz'
  TRAIN_LABELS = 'emnist-letters-train-labels-idx1-ubyte.gz'
  TEST_IMAGES = 'emnist-letters-test-images-idx3-ubyte.gz'
  TEST_LABELS = 'emnist-letters-test-labels-idx1-ubyte.gz'

  local_file = TRAIN_IMAGES #base.maybe_download(TRAIN_IMAGES, train_dir,
                                   #SOURCE_URL + TRAIN_IMAGES)
  with open(local_file, 'rb') as f:
    train_images = extract_images(f)

  local_file = TRAIN_LABELS #base.maybe_download(TRAIN_LABELS, train_dir,
                                   #SOURCE_URL + TRAIN_LABELS)
  with open(local_file, 'rb') as f:
    train_labels = extract_labels(f, one_hot=one_hot)

  train_labels, train_images = remove_data(train_labels, train_images)
  
  local_file = TEST_IMAGES #base.maybe_download(TEST_IMAGES, train_dir,
                                  # SOURCE_URL + TEST_IMAGES)
  with open(local_file, 'rb') as f:
    test_images = extract_images(f)

  local_file = TEST_LABELS #base.maybe_download(TEST_LABELS, train_dir,
                                   #SOURCE_URL + TEST_LABELS)
  with open(local_file, 'rb') as f:
    test_labels = extract_labels(f, one_hot=one_hot)

  test_labels, test_images = remove_data(test_labels, test_images)
  
  if not 0 <= validation_size <= len(train_images):
    raise ValueError(
        'Validation size should be between 0 and {}. Received: {}.'
        .format(len(train_images), validation_size))

  validation_images = train_images[:validation_size]
  validation_labels = train_labels[:validation_size]
  train_images = train_images[validation_size:]
  train_labels = train_labels[validation_size:]

  train = DataSet(train_images, train_labels, dtype=dtype, reshape=reshape)
  validation = DataSet(validation_images,
                       validation_labels,
                       dtype=dtype,
                       reshape=reshape)
  test = DataSet(test_images, test_labels, dtype=dtype, reshape=reshape)

  return base.Datasets(train=train, validation=validation, test=test)


def load_mnist(train_dir='MNIST-data'):
  return read_data_sets(train_dir)

In [6]:
#Importing emnist dataset from files
#NOTE: load dataset (gzip one) from https://www.nist.gov/node/1298471/emnist-dataset
emnist = read_data_sets("/", one_hot=True) 
print(len(emnist.test.images))
print(len(emnist.train.images))

Extracting emnist-letters-train-images-idx3-ubyte.gz
Extracting emnist-letters-train-labels-idx1-ubyte.gz
Extracting emnist-letters-test-images-idx3-ubyte.gz
Extracting emnist-letters-test-labels-idx1-ubyte.gz
8000
43000


In [0]:
#########Andrej's method taken from: http://blog.aloni.org/posts/backprop-with-tensorflow/ ##########
middle = 30
num_out = 10

a_0 = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, num_out])

### Comment out whichever u are not using
w_1 = tf.Variable(tf.truncated_normal([784, middle])) #tf.Variable(w1_file) 
b_1 = tf.Variable(tf.truncated_normal([1, middle]))
w_2 = tf.Variable(w2_file) #tf.Variable(tf.truncated_normal([middle, num_out])) #tf.Variable(w2_file)
b_2 = tf.Variable(tf.truncated_normal([1, num_out]))


are = w_2
def sigma(x):
    return tf.div(tf.constant(1.0),
                  tf.add(tf.constant(1.0), tf.exp(tf.negative(x))))
  
  
z_1 = tf.add(tf.matmul(a_0, w_1), b_1)
a_1 = sigma(z_1)
z_2 = tf.add(tf.matmul(a_1, w_2), b_2)
a_2 = sigma(z_2)


diff = tf.subtract(a_2, y)


def sigmaprime(x):
    return tf.multiply(sigma(x), tf.subtract(tf.constant(1.0), sigma(x)))
  
  
d_z_2 = tf.multiply(diff, sigmaprime(z_2))
d_b_2 = d_z_2
d_w_2 = tf.matmul(tf.transpose(a_1), d_z_2)

d_a_1 = tf.matmul(d_z_2, tf.transpose(w_2))
d_z_1 = tf.multiply(d_a_1, sigmaprime(z_1))
d_b_1 = d_z_1
d_w_1 = tf.matmul(tf.transpose(a_0), d_z_1)


eta = tf.constant(0.5)
step = [
    tf.assign(w_1,
            tf.subtract(w_1, tf.multiply(eta, d_w_1)))
  , tf.assign(b_1,
            tf.subtract(b_1, tf.multiply(eta,
                               tf.reduce_mean(d_b_1, axis=[0]))))
  , tf.assign(w_2,
            tf.subtract(w_2, tf.multiply(eta, d_w_2)))
  , tf.assign(b_2,
            tf.subtract(b_2, tf.multiply(eta,
                               tf.reduce_mean(d_b_2, axis=[0]))))
]


# loss functions 
acct_mat = tf.equal(tf.argmax(a_2, 1), tf.argmax(y, 1))
acct_res = tf.reduce_sum(tf.cast(acct_mat, tf.float32))


#initialize some variables
batch_size = 10
total_test_size = len(emnist.test.images)
epoch = range(300)
test_accuracy = []
train_accuracy = []

#start session
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

### train the model
for j in epoch:

  for i in range(total_test_size//batch_size):
      batch_xs, batch_ys = emnist.train.next_batch(batch_size)
      sess.run(step, feed_dict = {a_0: batch_xs,
                                  y : batch_ys})
  ### test the accuracy 
  test_acc = sess.run(acct_res, feed_dict =
                          {a_0: emnist.test.images[:1000],
                           y : emnist.test.labels[:1000]})/1000
  train_acc = sess.run(acct_res, feed_dict =
                          {a_0: emnist.train.images[:1000],
                           y : emnist.train.labels[:1000]})/1000
  test_accuracy.append(test_acc)
  train_accuracy.append(train_acc)
  print(
            "Iteration",
            str(j),
            "\t| test accuracy =",
            str(test_acc),
            "\t| train accuracy =",
            str(train_acc)
            )
  
#close the session  
sess.close()

In [0]:
#plot 
plt.figure(figsize=(12, 6))
#plt.title('EMNIST-letters using inital weights from normal distribution')
plt.title('10 EMNIST-letters using second layer of weights from MNIST-digits')
plt.plot(epoch, test_accuracy, label='test')
plt.plot(epoch, train_accuracy, label='train')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()



In [0]:
print(train_accuracy) 
print(test_accuracy)

In [0]:
test_accuracy_w1w2 = [0.042, 0.104, 0.159, 0.152, 0.146, 0.153, 0.149, 0.163, 0.159, 0.163, 0.162, 0.173, 0.145, 0.166, 0.164, 0.152, 0.172, 0.145, 0.181, 0.173, 0.185, 0.227, 0.253, 0.814, 0.699, 0.715, 0.812, 0.814, 0.764, 0.797, 0.816, 0.74, 0.82, 0.824, 0.769, 0.855, 0.833, 0.749, 0.834, 0.808, 0.835, 0.889, 0.831, 0.818, 0.817, 0.868, 0.823, 0.88, 0.844, 0.841, 0.849, 0.864, 0.892, 0.872, 0.88, 0.786, 0.889, 0.834, 0.818, 0.849, 0.858, 0.866, 0.834, 0.836, 0.833, 0.836, 0.859, 0.89, 0.833, 0.85, 0.835, 0.835, 0.836, 0.87, 0.845, 0.852, 0.895, 0.808, 0.835, 0.854, 0.803, 0.857, 0.878, 0.782, 0.865, 0.856, 0.844, 0.869, 0.844, 0.806, 0.822, 0.871, 0.778, 0.867, 0.869, 0.887, 0.845, 0.916, 0.875, 0.89, 0.844, 0.821, 0.831, 0.825, 0.881, 0.857, 0.872, 0.853, 0.831, 0.838, 0.861, 0.851, 0.849, 0.838, 0.855, 0.845, 0.83, 0.81, 0.82, 0.871, 0.865, 0.848, 0.814, 0.829, 0.83, 0.85, 0.83, 0.847, 0.823, 0.877, 0.872, 0.863, 0.816, 0.847, 0.791, 0.887, 0.851, 0.865, 0.858, 0.873, 0.831, 0.861, 0.852, 0.873, 0.872, 0.865, 0.881, 0.816, 0.848, 0.809, 0.885, 0.843, 0.88, 0.824, 0.838, 0.881, 0.859, 0.82, 0.873, 0.845, 0.87, 0.859, 0.886, 0.878, 0.844, 0.892, 0.857, 0.828, 0.838, 0.889, 0.852, 0.872, 0.846, 0.875, 0.845, 0.858, 0.862, 0.875, 0.86, 0.865, 0.877, 0.866, 0.873, 0.842, 0.86, 0.846, 0.868, 0.893, 0.855, 0.838, 0.813, 0.852, 0.846, 0.907, 0.84, 0.827, 0.828, 0.885, 0.896, 0.864, 0.844, 0.829, 0.853, 0.85, 0.866, 0.857, 0.856, 0.85, 0.863, 0.822, 0.861, 0.87, 0.873, 0.875, 0.849, 0.856, 0.825, 0.868, 0.863, 0.868, 0.864, 0.84, 0.875, 0.869, 0.837, 0.845, 0.875, 0.856, 0.841, 0.852, 0.836, 0.852, 0.853, 0.868, 0.831, 0.83, 0.836, 0.867, 0.88, 0.877, 0.874, 0.872, 0.86, 0.839, 0.871, 0.822, 0.852, 0.816, 0.869, 0.895, 0.865, 0.867, 0.852, 0.854, 0.814, 0.871, 0.826, 0.862, 0.87, 0.862, 0.847, 0.882, 0.858, 0.855, 0.873, 0.842, 0.842, 0.872, 0.832, 0.864, 0.863, 0.875, 0.901, 0.833, 0.849, 0.867, 0.862, 0.867, 0.854, 0.885, 0.834, 0.861, 0.852, 0.867, 0.858, 0.844, 0.841, 0.863, 0.874, 0.868, 0.849, 0.866, 0.871, 0.856, 0.864, 0.84, 0.864, 0.861, 0.854, 0.854]
train_accuracy_w1w2 = [0.187, 0.251, 0.319, 0.341, 0.378, 0.422, 0.426, 0.416, 0.422, 0.414, 0.437, 0.435, 0.435, 0.469, 0.512, 0.513, 0.508, 0.506, 0.516, 0.52, 0.52, 0.525, 0.541, 0.585, 0.574, 0.591, 0.618, 0.608, 0.6, 0.601, 0.604, 0.597, 0.61, 0.6, 0.596, 0.599, 0.603, 0.617, 0.614, 0.607, 0.614, 0.605, 0.606, 0.608, 0.612, 0.613, 0.606, 0.606, 0.624, 0.623, 0.628, 0.629, 0.614, 0.627, 0.617, 0.615, 0.625, 0.621, 0.611, 0.605, 0.609, 0.607, 0.6, 0.6, 0.619, 0.618, 0.619, 0.6, 0.607, 0.654, 0.628, 0.651, 0.648, 0.643, 0.638, 0.627, 0.619, 0.623, 0.622, 0.626, 0.635, 0.623, 0.628, 0.622, 0.624, 0.615, 0.628, 0.634, 0.632, 0.633, 0.629, 0.654, 0.642, 0.65, 0.637, 0.649, 0.636, 0.627, 0.627, 0.625, 0.618, 0.634, 0.622, 0.629, 0.637, 0.64, 0.684, 0.683, 0.701, 0.701, 0.706, 0.695, 0.713, 0.702, 0.694, 0.701, 0.706, 0.675, 0.723, 0.722, 0.729, 0.722, 0.724, 0.714, 0.723, 0.721, 0.725, 0.724, 0.718, 0.744, 0.736, 0.741, 0.733, 0.738, 0.699, 0.705, 0.686, 0.7, 0.701, 0.722, 0.716, 0.718, 0.71, 0.7, 0.707, 0.7, 0.702, 0.691, 0.695, 0.696, 0.751, 0.755, 0.754, 0.747, 0.737, 0.737, 0.725, 0.726, 0.727, 0.731, 0.723, 0.734, 0.729, 0.736, 0.725, 0.73, 0.723, 0.718, 0.722, 0.713, 0.708, 0.711, 0.723, 0.74, 0.726, 0.739, 0.732, 0.733, 0.72, 0.737, 0.73, 0.731, 0.729, 0.728, 0.716, 0.728, 0.722, 0.731, 0.733, 0.744, 0.737, 0.742, 0.737, 0.712, 0.717, 0.714, 0.721, 0.725, 0.727, 0.719, 0.723, 0.722, 0.708, 0.713, 0.73, 0.73, 0.721, 0.724, 0.726, 0.718, 0.713, 0.713, 0.699, 0.707, 0.705, 0.726, 0.725, 0.729, 0.727, 0.716, 0.708, 0.696, 0.705, 0.695, 0.694, 0.733, 0.732, 0.734, 0.721, 0.73, 0.726, 0.748, 0.748, 0.739, 0.746, 0.738, 0.734, 0.728, 0.731, 0.723, 0.722, 0.729, 0.722, 0.717, 0.717, 0.703, 0.719, 0.725, 0.731, 0.731, 0.722, 0.719, 0.743, 0.736, 0.733, 0.746, 0.734, 0.738, 0.733, 0.73, 0.729, 0.725, 0.732, 0.714, 0.712, 0.718, 0.721, 0.716, 0.705, 0.715, 0.718, 0.714, 0.711, 0.712, 0.772, 0.769, 0.768, 0.768, 0.769, 0.74, 0.745, 0.739, 0.728, 0.733, 0.753, 0.746, 0.743, 0.744, 0.748, 0.741, 0.728, 0.734, 0.733, 0.728, 0.72, 0.735, 0.741, 0.738, 0.736, 0.742]
train_accuracy_w1=[0.643, 0.682, 0.733, 0.766, 0.774, 0.805, 0.819, 0.84, 0.818, 0.841, 0.826, 0.828, 0.815, 0.821, 0.81, 0.822, 0.876, 0.862, 0.86, 0.868, 0.874, 0.86, 0.864, 0.871, 0.857, 0.861, 0.896, 0.88, 0.881, 0.878, 0.871, 0.864, 0.907, 0.9, 0.888, 0.893, 0.879, 0.9, 0.885, 0.877, 0.873, 0.873, 0.868, 0.871, 0.862, 0.879, 0.865, 0.862, 0.888, 0.876, 0.871, 0.867, 0.881, 0.894, 0.891, 0.882, 0.891, 0.894, 0.888, 0.892, 0.888, 0.88, 0.884, 0.887, 0.892, 0.882, 0.882, 0.882, 0.887, 0.917, 0.91, 0.92, 0.914, 0.902, 0.891, 0.897, 0.899, 0.897, 0.891, 0.883, 0.907, 0.899, 0.897, 0.889, 0.893, 0.88, 0.901, 0.886, 0.898, 0.89, 0.893, 0.914, 0.897, 0.905, 0.879, 0.893, 0.898, 0.894, 0.889, 0.89, 0.893, 0.887, 0.907, 0.877, 0.895, 0.889, 0.88, 0.914, 0.914, 0.909, 0.901, 0.895, 0.913, 0.9, 0.89, 0.895, 0.895, 0.891, 0.909, 0.913, 0.906, 0.907, 0.905, 0.903, 0.892, 0.898, 0.889, 0.897, 0.896, 0.897, 0.899, 0.899, 0.9, 0.897, 0.894, 0.897, 0.899, 0.902, 0.893, 0.92, 0.908, 0.903, 0.909, 0.909, 0.904, 0.915, 0.908, 0.911, 0.912, 0.915, 0.93, 0.922, 0.92, 0.908, 0.912, 0.924, 0.91, 0.912, 0.914, 0.911, 0.907, 0.902, 0.886, 0.903, 0.906, 0.906, 0.916, 0.916, 0.919, 0.92, 0.92, 0.913, 0.913, 0.903, 0.898, 0.911, 0.904, 0.931, 0.919, 0.917, 0.916, 0.933, 0.91, 0.918, 0.902, 0.91, 0.907, 0.907, 0.911, 0.91, 0.912, 0.906, 0.907, 0.922, 0.918, 0.913, 0.923, 0.921, 0.927, 0.918, 0.915, 0.911, 0.904, 0.915, 0.934, 0.92, 0.928, 0.924, 0.918, 0.929, 0.934, 0.907, 0.912, 0.925, 0.914, 0.905, 0.914, 0.909, 0.916, 0.909, 0.916, 0.914, 0.906, 0.91, 0.923, 0.907, 0.911, 0.909, 0.907, 0.912, 0.914, 0.91, 0.903, 0.899, 0.901, 0.904, 0.914, 0.913, 0.919, 0.908, 0.912, 0.923, 0.912, 0.926, 0.929, 0.92, 0.915, 0.905, 0.909, 0.905, 0.896, 0.895, 0.939, 0.928, 0.918, 0.93, 0.913, 0.92, 0.922, 0.921, 0.914, 0.911, 0.913, 0.899, 0.89, 0.892, 0.89, 0.882, 0.927, 0.925, 0.914, 0.918, 0.914, 0.911, 0.929, 0.927, 0.907, 0.919, 0.917, 0.92, 0.923, 0.906, 0.914, 0.928, 0.919, 0.898, 0.911, 0.902, 0.901, 0.907, 0.909, 0.923, 0.923, 0.909, 0.913, 0.928, 0.917, 0.917, 0.916, 0.915]
test_accuracy_w1=[0.729, 0.56, 0.663, 0.764, 0.714, 0.778, 0.788, 0.804, 0.765, 0.79, 0.849, 0.815, 0.811, 0.672, 0.781, 0.83, 0.854, 0.786, 0.798, 0.887, 0.84, 0.779, 0.789, 0.868, 0.837, 0.822, 0.841, 0.824, 0.867, 0.809, 0.817, 0.738, 0.844, 0.802, 0.845, 0.836, 0.848, 0.817, 0.825, 0.805, 0.816, 0.86, 0.829, 0.819, 0.873, 0.838, 0.818, 0.862, 0.813, 0.858, 0.803, 0.823, 0.84, 0.857, 0.849, 0.852, 0.85, 0.864, 0.863, 0.851, 0.859, 0.835, 0.864, 0.875, 0.846, 0.82, 0.849, 0.86, 0.832, 0.864, 0.89, 0.849, 0.858, 0.878, 0.852, 0.793, 0.878, 0.831, 0.86, 0.873, 0.845, 0.844, 0.881, 0.855, 0.868, 0.824, 0.864, 0.85, 0.874, 0.876, 0.877, 0.873, 0.855, 0.868, 0.858, 0.838, 0.877, 0.866, 0.842, 0.846, 0.83, 0.815, 0.83, 0.9, 0.851, 0.853, 0.827, 0.837, 0.858, 0.857, 0.847, 0.816, 0.866, 0.861, 0.839, 0.85, 0.821, 0.851, 0.847, 0.842, 0.847, 0.874, 0.829, 0.883, 0.871, 0.879, 0.897, 0.858, 0.842, 0.845, 0.859, 0.858, 0.837, 0.863, 0.891, 0.863, 0.886, 0.841, 0.846, 0.892, 0.866, 0.888, 0.874, 0.883, 0.849, 0.856, 0.841, 0.841, 0.859, 0.855, 0.859, 0.881, 0.846, 0.883, 0.862, 0.873, 0.876, 0.854, 0.863, 0.863, 0.882, 0.862, 0.877, 0.88, 0.89, 0.886, 0.858, 0.883, 0.843, 0.869, 0.857, 0.86, 0.861, 0.857, 0.841, 0.882, 0.844, 0.879, 0.86, 0.878, 0.875, 0.865, 0.822, 0.843, 0.888, 0.877, 0.84, 0.882, 0.846, 0.865, 0.882, 0.886, 0.878, 0.86, 0.865, 0.834, 0.863, 0.875, 0.863, 0.846, 0.868, 0.862, 0.846, 0.843, 0.858, 0.85, 0.87, 0.867, 0.881, 0.852, 0.874, 0.85, 0.865, 0.885, 0.853, 0.871, 0.855, 0.848, 0.873, 0.844, 0.846, 0.843, 0.854, 0.858, 0.877, 0.875, 0.856, 0.853, 0.866, 0.887, 0.894, 0.825, 0.875, 0.882, 0.825, 0.879, 0.886, 0.873, 0.852, 0.844, 0.886, 0.843, 0.908, 0.852, 0.832, 0.849, 0.876, 0.851, 0.888, 0.885, 0.808, 0.893, 0.836, 0.826, 0.872, 0.891, 0.861, 0.892, 0.866, 0.853, 0.866, 0.856, 0.871, 0.858, 0.895, 0.881, 0.854, 0.853, 0.879, 0.882, 0.882, 0.871, 0.867, 0.883, 0.877, 0.867, 0.848, 0.868, 0.879, 0.858, 0.882, 0.827, 0.869, 0.879, 0.879, 0.878, 0.832, 0.89, 0.845, 0.858, 0.826, 0.873, 0.868, 0.823, 0.845, 0.873, 0.886, 0.881, 0.865, 0.867]
train_accuracy_w2 = 
test_accuracy_w2 = 
train_accuracy_normal =
test_accuracy_normal = 



In [0]:
####get train_accuracy_w1 and normal from text file
#plot 
plt.figure(figsize=(12, 6))
plt.title('EMNIST-letters test data using transferred initial weights vs. normal distribution (zoomed)')
plt.plot(epoch,test_accuracy_normal, label='normal dist')
plt.plot(epoch,test_accuracy_usingw1, label='transferred weights')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()

